In [2]:
#Importation of the pandas library 

import pandas as pd

In [3]:
pd.set_option('max_columns', None)

#bookings

df_bookings = pd.read_csv("bookings.csv.bz2", compression='bz2', nrows = 10000, sep = '^')

bookings_head20 = df_bookings.head(20)
bookings_info = df_bookings.info()
bookings_columns = df_bookings.columns

#searches

df_searches = pd.read_csv("searches.csv.bz2", compression='bz2', nrows = 10000, sep = '^')
searches_head20 = df_searches.head(20)
searches_info = df_searches.info()
searches_columns = df_searches.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   act_date             10000 non-null  object
 1   source               10000 non-null  object
 2   pos_ctry             10000 non-null  object
 3   pos_iata             10000 non-null  object
 4   pos_oid              10000 non-null  object
 5   rloc                 10000 non-null  object
 6   cre_date             10000 non-null  object
 7   duration             10000 non-null  int64 
 8   distance             10000 non-null  int64 
 9   dep_port             10000 non-null  object
 10  dep_city             10000 non-null  object
 11  dep_ctry             10000 non-null  object
 12  arr_port             10000 non-null  object
 13  arr_city             10000 non-null  object
 14  arr_ctry             10000 non-null  object
 15  lst_port             10000 non-null  object
 16  lst_c

First, let's take a look at the fields that are in both datasets to check by which labels we will pair up:

    a) Origin and destination airports already suggested in the wording of the exercise.
    
    b) Same day : cre_date(bookings) and Date(searches).
    
    c) I will keep the Seg1BookingCode from the researches dataset in case it's useful.

In [3]:
df_bookings.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

##### Booking dataset preparation

In [5]:
# Importing only the key data

file = pd.read_csv("bookings.csv.bz2", compression='bz2', nrows = 5, sep = '^')
filecolumns = file.columns

#Create a list with the headers without empty spaces

list_columns = list(filecolumns)
fixed_columns = []
for col in list_columns:
    fixed_columns.append(col.strip())

cols = ['act_date','cre_date','dep_port','arr_port','brd_time','off_time','oid']
df_bookings = pd.read_csv("bookings.csv.bz2", compression='bz2', nrows = 10000, sep = '^', skiprows = 1, usecols = cols, names = fixed_columns, parse_dates = ['act_date','cre_date','brd_time','off_time'])

    # cleaning dataset empty spaces
for i, r in enumerate(df_bookings['arr_port']):  
    r1 = r.strip()
    df_bookings.at[i, 'arr_port'] = r1
    
for i, r in enumerate(df_bookings['dep_port']):  
    r1 = r.strip()
    df_bookings.at[i, 'dep_port'] = r1
    
df_bookings['CommonKey'] = df_bookings['act_date'].astype(str) + ' ' + df_bookings['dep_port'] + ' ' + df_bookings['arr_port'] # Creation of a common key as a column
df_bookings['Counter'] = 1
df_bookings.head()

,act_date,cre_date,dep_port,arr_port,brd_time,off_time,oid,CommonKey,Counter
0,2013-03-05,2013-02-22,ZRH,LHR,2013-03-07 08:50:00,2013-03-07 11:33:37,NULL,2013-03-05 ZRH LHR,1
1,2013-03-26,2013-03-26,SAL,CLT,2013-04-12 13:04:00,2013-04-12 22:05:40,NULL,2013-03-26 SAL CLT,1
2,2013-03-26,2013-03-26,SAL,CLT,2013-07-15 07:00:00,2013-07-15 11:34:51,NULL,2013-03-26 SAL CLT,1
3,2013-03-26,2013-03-26,AKL,SVO,2013-04-24 23:59:00,2013-04-25 16:06:31,SYDA82546,2013-03-26 AKL SVO,1
4,2013-03-26,2013-03-26,AKL,SVO,2013-05-14 20:15:00,2013-05-16 10:44:50,SYDA82546,2013-03-26 AKL SVO,1


##### Searches dataset preparation

In [6]:
# Importing only the key data

col_searches = ['Date','Origin','Destination','Seg1BookingCode']
df_searches = pd.read_csv("searches.csv.bz2", compression='bz2', nrows = 10000, sep = '^', usecols = col_searches, parse_dates = ['Date'])
df_searches['CommonKey'] = df_searches['Date'].astype(str) + ' ' + df_searches['Origin'] + ' ' + df_searches['Destination'] # Creation of a common key as a column
df_searches.head()

,Date,Origin,Destination,Seg1BookingCode,CommonKey
0,2013-01-01,TXL,AUH,NaN,2013-01-01 TXL AUH
1,2013-01-01,ATH,MIL,NaN,2013-01-01 ATH MIL
2,2013-01-01,ICT,SFO,NaN,2013-01-01 ICT SFO
3,2013-01-01,RNB,ARN,W,2013-01-01 RNB ARN
4,2013-01-01,OSL,MAD,NaN,2013-01-01 OSL MAD


#### Testing the merge of both datasets: I will work with two small databases to check that will work. So that :

    a) I will select the first 10K rows from both databases
    b) To simplify I will consider that only Origin and Destination airports are the fields to check if a search ends up with a booking ( Date will be needed when I work with the complete databases)
    c) From searches I will pick only ones where Origin = LAX
    d) From bookings I will select only yhe ones where Origin = LAX and Destination = CDG
    e) I must have the original searches dataset indicating when Origin = LAX and Destination = CDG

In [8]:
# Selection of the 10K first rows filtering by Origin : LAX ---> I REFUSED THE IDEA OF CREATING A COMMONKEY!!!!

df_searches['CommonKey2'] = df_searches['Origin'] + ' ' + df_searches['Destination']
test_searches = df_searches[(df_searches['Origin'] == 'LAX')]
test_searches

,Date,Origin,Destination,Seg1BookingCode,CommonKey,CommonKey2
174,2013-01-01,LAX,BJX,NaN,2013-01-01 LAX BJX,LAX BJX
286,2013-01-01,LAX,CDG,NaN,2013-01-01 LAX CDG,LAX CDG
372,2013-01-01,LAX,MEL,NaN,2013-01-01 LAX MEL,LAX MEL
386,2013-01-01,LAX,TLV,NaN,2013-01-01 LAX TLV,LAX TLV
438,2013-01-01,LAX,MEX,NaN,2013-01-01 LAX MEX,LAX MEX
...,...,...,...,...,...,...
9754,2013-01-10,LAX,MEX,NaN,2013-01-10 LAX MEX,LAX MEX
9775,2013-01-10,LAX,DXB,NaN,2013-01-10 LAX DXB,LAX DXB
9808,2013-01-10,LAX,SAL,C,2013-01-10 LAX SAL,LAX SAL
9834,2013-01-10,LAX,LHR,NaN,2013-01-10 LAX LHR,LAX LHR


In [9]:
# Selection of the 10K first rows filtering by Origin : LAX and Destination: GDG

df_bookings['CommonKey2'] = df_bookings['dep_port'] + ' ' + df_bookings['arr_port']
test_bookings = df_bookings[(df_bookings['dep_port'] == 'LAX') & (df_bookings['arr_port'] == 'CDG')]
test_bookings = test_bookings.groupby('CommonKey2')['Counter'].sum().reset_index()
test_bookings

,CommonKey2,Counter
0,LAX CDG,4


In [10]:
# Update searches dataframe ( Origin = LAX) with booking dataframe ( Origin = LAX & Destination = CGD)

# If it works I will get a 139 rows dataframe with a new Column Counter that will be 1 when Destination = CDG

test_result = test_searches.merge(test_bookings,how='left', on ='CommonKey2',indicator = False)
test_result['Counter'] = test_result['Counter']/test_result['Counter']
test_result['Counter'] = test_result['Counter'].fillna(0)
test_result.Counter.sum()


3.0

##### I am getting a 139 rows dataframe. But this time with the additional Column 'Counter' that will be different to zero when matches up with the bookings dataframe. The total matches is equal to 3

In [11]:
df_searches['CommonKey2'] = df_searches['Origin'] + ' ' + test_searches['Destination']
test_searches = df_searches[(df_searches['Origin'] == 'LAX') & (df_searches['Destination'] == 'CDG')]
test_searches

,Date,Origin,Destination,Seg1BookingCode,CommonKey,CommonKey2
286,2013-01-01,LAX,CDG,NaN,2013-01-01 LAX CDG,LAX CDG
6119,2013-01-07,LAX,CDG,NaN,2013-01-07 LAX CDG,LAX CDG
9669,2013-01-10,LAX,CDG,NaN,2013-01-10 LAX CDG,LAX CDG


##### Looks like it works

## Initiating the solution of the exercise

##### Fixed booking headers

In [12]:
file = pd.read_csv("bookings.csv.bz2", compression='bz2', nrows = 5, sep = '^')
filecolumns = file.columns

#Create a list with the headers without empty spaces

list_columns = list(filecolumns)
fixed_columns = []
for col in list_columns:
    fixed_columns.append(col.strip())

##### Simplified booking dataset ( 1 unique line x date x origin x destination)

In [13]:
df_ini = pd.DataFrame() # Initialisation empty dataframe

n_chunks = 0 # Initialisation counter of chunks

rows = 0 # Initialisation counter of rows

for test in pd.read_csv("bookings.csv.bz2", compression='bz2', sep = '^', skiprows = 1, names = fixed_columns, parse_dates = ['cre_date'],usecols = ['cre_date','dep_port', 'arr_port', 'pax'],chunksize = 250000):
    
    rows += len(test) # Adding up the number of lines of the chunktest['pax'] = test['pax'].fillna(0) # replacing Nan values from the column pax by 0
    
    n_chunks += 1 # Adding up a new chunk
    
    test = test[test['pax'] > 0 ] # Selection only the rows where it's indicated at least one passenger
    
    test['Counter'] = 1

    try:
        
        test['year'] = test['cre_date'].dt.year
    
        test['month'] = test['cre_date'].dt.month
    
        test['day'] = test['cre_date'].dt.day

        arr_group = test.groupby(['month','day','dep_port','arr_port'])['Counter'].agg('sum').reset_index() # Group sum of passenger by arr_port
    
        for i, r in enumerate(arr_group['arr_port']):  # Eliminate white spaces from the column arr_port
        
            r1 = r.strip()
            arr_group.at[i, 'arr_port'] = r1
        
        for i, r in enumerate(arr_group['dep_port']):  # Eliminate white spaces from the column arr_port
        
            r1 = r.strip()
            arr_group.at[i, 'dep_port'] = r1   
    except:
        pass
    
    df_ini = df_ini.append(arr_group) # Adding in the dataset retrieved from the current chunk to the accumulated one.
    
    df_bookings_ini = df_ini.groupby(['month','day','dep_port','arr_port'])['Counter'].agg('sum').reset_index()# Group the accumulated results by arr_group
        
    print('Accumulated lines proccesed: ' + str(rows) + ' ( Number of chunks: '+ str(n_chunks) + ')')

Accumulated lines proccesed: 250000 ( Number of chunks: 1)
Accumulated lines proccesed: 500000 ( Number of chunks: 2)
Accumulated lines proccesed: 750000 ( Number of chunks: 3)
Accumulated lines proccesed: 1000000 ( Number of chunks: 4)
Accumulated lines proccesed: 1250000 ( Number of chunks: 5)
Accumulated lines proccesed: 1500000 ( Number of chunks: 6)
Accumulated lines proccesed: 1750000 ( Number of chunks: 7)
Accumulated lines proccesed: 2000000 ( Number of chunks: 8)
Accumulated lines proccesed: 2250000 ( Number of chunks: 9)
Accumulated lines proccesed: 2500000 ( Number of chunks: 10)
Accumulated lines proccesed: 2750000 ( Number of chunks: 11)
Accumulated lines proccesed: 3000000 ( Number of chunks: 12)
Accumulated lines proccesed: 3250000 ( Number of chunks: 13)
Accumulated lines proccesed: 3500000 ( Number of chunks: 14)
Accumulated lines proccesed: 3750000 ( Number of chunks: 15)
Accumulated lines proccesed: 4000000 ( Number of chunks: 16)
Accumulated lines proccesed: 4250000

##### Merging datasets

In [15]:
# Initiating items

df_searches_ini = pd.DataFrame()

n_chunks = 0

rows = 0
    
# Importation

for file in pd.read_csv("searches.csv.bz2", compression='bz2',sep = '^',chunksize = 1000000):
    
    #Counting rows and chunks 
    
    rows += len(file)
    
    n_chunks += 1
    
    # Dates

    try:
        
        file['Date'] = pd.to_datetime(file['Date'])

        file['month'] = file['Date'].dt.month

        file['day'] = file['Date'].dt.day
        
    

    # Mergin with the simplified bookings dataset

        searches_result = file.merge(df_bookings_ini,how = 'left', left_on =['month','day','Origin','Destination'],right_on = ['month','day','dep_port','arr_port'],indicator = False)
    
        searches_result['Counter'] = searches_result['Counter']/searches_result['Counter'] # Converting the Counter number into 1 when both datasets match up.
    
        searches_result['Counter'] = searches_result['Counter'].fillna(0) # Replacing Nan by 0.
    
    except:
        pass
    
    # Aggregating chunks to the database
    
    df_searches_ini = df_searches_ini.append(searches_result) # I add in the chunk to the dataset which is accumulating all the chunks.
    
    print('Accumulated lines proccesed: ' + str(rows) + ' ( Number of chunks: '+ str(n_chunks) + ')')

/Users/Dani/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Accumulated lines proccesed: 1000000 ( Number of chunks: 1)


/Users/Dani/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (40,41,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Accumulated lines proccesed: 2000000 ( Number of chunks: 2)
Accumulated lines proccesed: 3000000 ( Number of chunks: 3)
Accumulated lines proccesed: 4000000 ( Number of chunks: 4)
Accumulated lines proccesed: 5000000 ( Number of chunks: 5)
Accumulated lines proccesed: 6000000 ( Number of chunks: 6)
Accumulated lines proccesed: 7000000 ( Number of chunks: 7)
Accumulated lines proccesed: 8000000 ( Number of chunks: 8)
Accumulated lines proccesed: 9000000 ( Number of chunks: 9)
Accumulated lines proccesed: 10000000 ( Number of chunks: 10)
Accumulated lines proccesed: 11000000 ( Number of chunks: 11)
Accumulated lines proccesed: 12000000 ( Number of chunks: 12)
Accumulated lines proccesed: 13000000 ( Number of chunks: 13)


/Users/Dani/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Accumulated lines proccesed: 14000000 ( Number of chunks: 14)
Accumulated lines proccesed: 15000000 ( Number of chunks: 15)
Accumulated lines proccesed: 16000000 ( Number of chunks: 16)
Accumulated lines proccesed: 17000000 ( Number of chunks: 17)
Accumulated lines proccesed: 18000000 ( Number of chunks: 18)
Accumulated lines proccesed: 19000000 ( Number of chunks: 19)
Accumulated lines proccesed: 20000000 ( Number of chunks: 20)
Accumulated lines proccesed: 20390198 ( Number of chunks: 21)


In [16]:
df_searches_ini.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,month,day,dep_port,arr_port,Counter
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0.0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,FRA,1,1,NaN,NaN,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0.0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,KIV,1,1,NaN,NaN,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0.0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,NYC,1,1,NaN,NaN,0.0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,ARN,2013-01-02,DU,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0.0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,STO,1,1,NaN,NaN,0.0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,MAD,2013-03-22,NaN,NaN,MAD,OSL,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0.0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,OSL,1,1,NaN,NaN,0.0


##### Exporting_output

In [24]:
df_searches_final = df_searches_ini.drop(['month','day','dep_port','arr_port'], axis = 1)

In [25]:
df_searches_final.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,Counter
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0.0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,FRA,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0.0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,KIV,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0.0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,NYC,0.0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,ARN,2013-01-02,DU,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0.0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,STO,0.0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,MAD,2013-03-22,NaN,NaN,MAD,OSL,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0.0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,OSL,0.0


In [32]:
df_searches_final.to_csv('DS_Challenge_Exercise4.csv.bz2', sep = '^', compression ='bz2',index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
